In [1]:
! pip install scikit-learn-intelex

In [2]:
from sklearnex import patch_sklearn
patch_sklearn()

import json
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import pickle
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report


PATH="/kaggle/input/webis-clickbait-22/"
sep='[SEP]'

label_encoder = LabelEncoder()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:

def get_split_data(file_path,if_train=True):
    global label_encoder
    data=pd.read_json(file_path,lines=True,encoding='utf-8')

    y = data["tags"]
    Y = [i[0] for i in y]
    print("len of Y: ",len(Y))
    if if_train:
        Y = label_encoder.fit_transform(Y)
    else:
        Y = label_encoder.transform(Y)

    new_data=data[['postText','targetParagraphs','targetTitle','targetDescription','targetKeywords']]
    dic=new_data.to_dict(orient='records')

    desccount=0
    keycount=0
    for i in dic:
        # print(i)
        i['postText']='postText: '+i['postText'][0]
        targetparah=" ".join(i['targetParagraphs'])
        i['targetParagraphs']='targetParagraphs: '+targetparah
        i['targetTitle']='targetTitle: '+i['targetTitle']
        targetDescription=i['targetDescription']
        if(targetDescription is None or len(targetDescription)==0):
            targetDescription='none'
            desccount+=1
        i['targetDescription']='targetDescription: '+targetDescription
        targetkey=i['targetKeywords']
        if(targetkey is None or len(targetkey)==0):
            targetkey='none'
            keycount+=1
        i['targetKeywords']='targetKeywords: '+targetkey
        i['text']=i['postText']+sep+i['targetParagraphs']+sep+i['targetTitle']+sep+i['targetDescription']+sep+i['targetKeywords']
        del i['postText']
        del i['targetParagraphs']
        del i['targetTitle']
        del i['targetDescription']
        del i['targetKeywords']

    print("desccount: ",desccount)
    print("keycount: ",keycount)
    X = [i['text'] for i in dic]
    X=np.array(X)
#     return X[0:100],Y[0:100]
    return X,Y


In [4]:

x_train,y_train = get_split_data(PATH+'train.jsonl')
x_test,y_test = get_split_data(PATH+'test.jsonl',False)
x_val,y_val = get_split_data(PATH+'validation.jsonl',False)


len of Y:  3200
desccount:  332
keycount:  1314
len of Y:  1000
desccount:  104
keycount:  414
len of Y:  800
desccount:  78
keycount:  330


In [5]:
# len(x_train),len(y_train),len(x_test),len(y_test),len(x_val),len(y_val),len(x_train[0]),len(y_train[0])
x_train.shape,y_train.shape,x_test.shape,y_test.shape,x_val.shape,y_val.shape,len(x_train[10])


((3200,), (3200,), (1000,), (1000,), (800,), (800,), 3518)

In [6]:
type(x_train),type(y_train),type(x_test),type(y_test),type(x_val),type(y_val),type(x_train[0]),type(y_train[0])

(numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.ndarray,
 numpy.str_,
 numpy.int64)

In [7]:

param_grid = {
    "vectorizer__stop_words": [None, "english"],
    'vectorizer__ngram_range': [(1, 1), (1, 2), (2, 2)],
    "model_logistic__C": [0.4, 0.6, 0.8, 1.0,1.5,2.0,2.5],
#     "model_logistic__multi_class":["auto", "ovr", "multinomial"]
}

tf_idf_pipeline = Pipeline([
    ('vectorizer', TfidfVectorizer()),
    ('model_logistic', LogisticRegression()),
])

grid_search = GridSearchCV(tf_idf_pipeline, param_grid, cv=10, n_jobs=-1,verbose=4)
grid_search.fit(x_train, y_train)

optimal_C = grid_search.best_params_["model_logistic__C"]
optimal_ngram = grid_search.best_params_["vectorizer__ngram_range"]
optimal_stop_words = grid_search.best_params_["vectorizer__stop_words"]
# optimal_multi_class = grid_search.best_params_["model_logistic__multi_class"]
print("Optimal value of the regularization constant (C) =", optimal_C)
print("Optimal value of ngram_range =", optimal_ngram)
print("Optimal value of stop_words =", optimal_stop_words)
# print("Optimal value of multi_class =", optimal_multi_class)


Fitting 10 folds for each of 42 candidates, totalling 420 fits


INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_mod

[CV 1/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=None;, score=0.525 total time=  10.3s
[CV 5/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=None;, score=0.506 total time=  10.1s
[CV 10/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=None;, score=0.534 total time=  10.4s
[CV 5/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=english;, score=0.481 total time=   9.2s
[CV 9/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=english;, score=0.512 total time=   9.2s
[CV 3/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 2), vectorizer__stop_words=None;, score=0.494 total time=  50.8s
[CV 5/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 2), vectorizer__stop_words=None;, score=0.512 total time=  49.3s
[CV 9/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 2), vectorizer__st

INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn


[CV 3/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=None;, score=0.516 total time=  10.7s
[CV 6/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=None;, score=0.572 total time=   9.8s
[CV 9/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=None;, score=0.559 total time=  11.0s
[CV 6/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=english;, score=0.581 total time=   9.1s
[CV 10/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=english;, score=0.525 total time=   9.4s
[CV 4/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 2), vectorizer__stop_words=None;, score=0.525 total time=  52.0s
[CV 6/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 2), vectorizer__stop_words=None;, score=0.569 total time=  53.5s
[CV 1/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 2), vectorizer__st

...

[CV 2/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=None;, score=0.537 total time=  10.9s
[CV 8/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=None;, score=0.469 total time=  10.0s
[CV 2/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=english;, score=0.519 total time=   9.0s
[CV 3/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=english;, score=0.516 total time=   9.1s
[CV 7/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=english;, score=0.509 total time=   9.1s
[CV 1/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 2), vectorizer__stop_words=None;, score=0.550 total time=  55.4s
[CV 7/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 2), vectorizer__stop_words=None;, score=0.500 total time=  52.7s
[CV 2/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 2), vectorizer__

INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn


[CV 4/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=None;, score=0.519 total time=  10.6s
[CV 7/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=None;, score=0.500 total time=  10.0s
[CV 1/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=english;, score=0.516 total time=   9.5s
[CV 4/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=english;, score=0.487 total time=   8.9s
[CV 8/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 1), vectorizer__stop_words=english;, score=0.463 total time=   9.1s
[CV 2/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 2), vectorizer__stop_words=None;, score=0.537 total time=  55.2s
[CV 8/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 2), vectorizer__stop_words=None;, score=0.444 total time=  50.3s
[CV 10/10] END model_logistic__C=0.4, vectorizer__ngram_range=(1, 2), vectorizer_

INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.linear

Optimal value of the regularization constant (C) = 2.5
Optimal value of ngram_range = (1, 2)
Optimal value of stop_words = None


In [8]:

vectorizer = TfidfVectorizer(ngram_range=optimal_ngram,stop_words=optimal_stop_words)
x_train =  vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)
x_val = vectorizer.transform(x_val)
print(x_train.shape)
print(x_test.shape)



INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


(3200, 763088)
(1000, 763088)


In [9]:

# Create and train logistic regression classifier
logistic_classifier = LogisticRegression(C=optimal_C, max_iter=1000,verbose=1)
logistic_classifier.fit(x_train, y_train)

# Predictions
y_train_pred = logistic_classifier.predict(x_train)
y_test_pred = logistic_classifier.predict(x_test)
y_val_pred = logistic_classifier.predict(x_val)

# Print metrics
print("Training data metrics:")
print(accuracy_score(y_train,y_train_pred))

print("Testing data accuracy:")
print(accuracy_score(y_test_pred,y_test))

print("Validation data accuracy:")
print(accuracy_score(y_val_pred,y_val))

# Save the model, label encoder, and vectorizer
pickle.dump((logistic_classifier, label_encoder, vectorizer), open('model_logistic.pkl', 'wb'))


INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =      2289267     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.51556D+03    |proj g|=  5.07667D+02


 This problem is unconstrained.



At iterate   50    f=  1.91852D+03    |proj g|=  8.83463D-01


INFO:sklearnex: sklearn.linear_model.LogisticRegression.fit: fallback to original Scikit-learn



           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
*****     87    101      1     0     0   1.031D-01   1.918D+03
  F =   1918.4302422995015     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


Training data metrics:
0.99625
Testing data accuracy:
0.553
Validation data accuracy:
0.55


In [10]:
print("Training data metrics:")
print(balanced_accuracy_score(y_train,y_train_pred))
print(f1_score(y_train_pred, y_train, average='macro'))
print(classification_report(y_train_pred, y_train))

print("Testing data metrics:")
print(balanced_accuracy_score(y_test_pred, y_test))
print(f1_score(y_test_pred, y_test, average='macro'))
print(classification_report(y_test_pred, y_test))

print("Validation data metrics:")
print(balanced_accuracy_score(y_val_pred, y_val))
print(f1_score(y_val_pred, y_val, average='macro'))
print(classification_report(y_val_pred, y_val))

Training data metrics:
0.9938661445003386
0.9952216047276625
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       551
           1       1.00      1.00      1.00      1278
           2       1.00      1.00      1.00      1371

    accuracy                           1.00      3200
   macro avg       0.99      1.00      1.00      3200
weighted avg       1.00      1.00      1.00      3200

Testing data metrics:
0.555373198230341
0.5190758995906054
              precision    recall  f1-score   support

           0       0.32      0.56      0.40        98
           1       0.60      0.56      0.58       429
           2       0.61      0.55      0.58       473

    accuracy                           0.55      1000
   macro avg       0.51      0.56      0.52      1000
weighted avg       0.57      0.55      0.56      1000

Validation data metrics:
0.5457955222554517
0.5141139356137648
              precision    recall  f1-score   support



In [11]:
model2,encoder,vectorizer = pickle.load(open("/kaggle/working/model_logistic.pkl","rb"))
y_train_pred = model2.predict(x_train)
y_test_pred = model2.predict(x_test)
y_val_pred = model2.predict(x_val)
print("Training data metrics:")
print(accuracy_score(y_train,y_train_pred))

print("Testing data accuracy:")
print(accuracy_score(y_test_pred,y_test))

print("Validation data accuracy:")
print(accuracy_score(y_val_pred,y_val))

INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU
INFO:sklearnex: sklearn.linear_model.LogisticRegression.predict: fallback to original Scikit-learn
INFO:sklearnex: sklearn.utils.validation._assert_all_finite: running accelerated version on CPU


Training data metrics:
0.99625
Testing data accuracy:
0.553
Validation data accuracy:
0.55


In [12]:
print("Training data metrics:")
print(balanced_accuracy_score(y_train,y_train_pred))
print(f1_score(y_train_pred, y_train, average='macro'))
print(classification_report(y_train_pred, y_train))

print("Testing data metrics:")
print(balanced_accuracy_score(y_test_pred, y_test))
print(f1_score(y_test_pred, y_test, average='macro'))
print(classification_report(y_test_pred, y_test))

print("Validation data metrics:")
print(balanced_accuracy_score(y_val_pred, y_val))
print(f1_score(y_val_pred, y_val, average='macro'))
print(classification_report(y_val_pred, y_val))

Training data metrics:
0.9938661445003386
0.9952216047276625
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       551
           1       1.00      1.00      1.00      1278
           2       1.00      1.00      1.00      1371

    accuracy                           1.00      3200
   macro avg       0.99      1.00      1.00      3200
weighted avg       1.00      1.00      1.00      3200

Testing data metrics:
0.555373198230341
0.5190758995906054
              precision    recall  f1-score   support

           0       0.32      0.56      0.40        98
           1       0.60      0.56      0.58       429
           2       0.61      0.55      0.58       473

    accuracy                           0.55      1000
   macro avg       0.51      0.56      0.52      1000
weighted avg       0.57      0.55      0.56      1000

Validation data metrics:
0.5457955222554517
0.5141139356137648
              precision    recall  f1-score   support

